# Evaluating a ResNet18 Model on Tiny ImageNet using Doleus
This notebook demonstrates the use of the Doleus library for evaluating a pretrained ResNet18 model on a subset of the Tiny ImageNet dataset.

In [26]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import zipfile
import urllib.request
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from doleus.utils.data import Task
from doleus.checks import Check, CheckSuite
from doleus.datasets import DoleusClassification

## Step 1: Download the Dataset

In [27]:
data_dir = "./tiny-imagenet-200"
zip_file = "tiny-imagenet-200.zip"
url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
model_checkpoint_url = "https://huggingface.co/zeyuanyin/tiny-imagenet/resolve/main/rn18_50ep/checkpoint.pth"
checkpoint_path = "./rn18_50ep.pth"

def download_tiny_imagenet(data_dir: str, zip_file: str, url: str) -> None:
    if not os.path.exists(data_dir):
        print("Tiny ImageNet not found. Downloading...")
        urllib.request.urlretrieve(url, zip_file)
        print("Download complete. Extracting...")
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(".")
        print("Extraction complete.")
        os.remove(zip_file)
    else:
        print("Tiny ImageNet is already set up.")

download_tiny_imagenet(data_dir, zip_file, url)

Tiny ImageNet is already set up.


## Step 2: Download the Model Checkpoint

In [28]:
if not os.path.exists(checkpoint_path):
    print("Downloading pretrained model checkpoint...")
    urllib.request.urlretrieve(model_checkpoint_url, checkpoint_path)
    print("Checkpoint downloaded.")

## Step 3: Define the Preprocessing

In [29]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Step 4: Load the Dataset

In [30]:
train_dataset = datasets.ImageFolder(
    root=os.path.join(data_dir, "train"), transform=transform)
subset_size = 100
indices = torch.randperm(len(train_dataset))[:subset_size]
subset = Subset(train_dataset, indices)
train_loader = DataLoader(subset, batch_size=8, shuffle=False)

## Step 5: Load the Model

In [31]:
model = torchvision.models.resnet18(num_classes=200)
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
model.maxpool = torch.nn.Identity()
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model_weights = checkpoint['model']
model.load_state_dict(model_weights)
model.eval()

/var/folders/tl/7nfbw2bn34v76r7t2lvxd0xw0000gn/T/ipykernel_47776/1991358245.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

## Step 6: Generate Predictions

In [32]:
predictions = []
with torch.no_grad():
    for images, lbls in train_loader:
        outputs = model(images)
        predictions.append(outputs)
predictions = torch.cat(predictions, dim=0)

## Step 7: Create Doleus Dataset

In [33]:
doleus_dataset = DoleusClassification(
    name="tiny_imagenet_subset",
    dataset=subset,
    task=Task.MULTICLASS.value,
    num_classes=200
)

Building GROUND TRUTHS for tiny_imagenet_subset: 100%|██████████| 100/100 [00:00<00:00, 3337.45it/s]


## Step 8: Add Model Predictions

In [34]:
doleus_dataset.add_model_predictions(
    predictions=predictions,
    model_id="resnet18",
)

## Step 9: Add Metadata

In [35]:
doleus_dataset.add_predefined_metadata("brightness")

Adding metadata 'brightness': 100%|██████████| 100/100 [00:00<00:00, 3479.71it/s]


## Step 10: Slicing

In [36]:
slice_bright = doleus_dataset.slice_by_percentile("brightness", ">=", 50)
slice_dim = doleus_dataset.slice_by_percentile("brightness", "<", 50)

## Step 11: Run Checks

In [37]:
check_suite = CheckSuite(name="test_brightness", checks=[
    Check(name="accuracy_bright", dataset=slice_bright, model_id="resnet18", metric="Accuracy", operator=">", value=0.5),
    Check(name="accuracy_dim", dataset=slice_dim, model_id="resnet18", metric="Accuracy", operator=">", value=0.5)
])
test_results = check_suite.run_all(show=True)

✅ test_brightness
    ✅ accuracy_bright                    0.87234 > 0.5      (Accuracy on tiny_imagenet_subset_brightness_ge_50)
    ✅ accuracy_dim                       0.88372 > 0.5      (Accuracy on tiny_imagenet_subset_brightness_lt_50)
